In [1]:
import fitz # import PyMuPDF
# import json
# import pandas as pd
import csv
import os
# import re

# import zipfile
import json
import pandas as pd

# from adobe.pdfservices.operation.auth.credentials import Credentials
# from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
# from adobe.pdfservices.operation.execution_context import ExecutionContext
# from adobe.pdfservices.operation.io.file_ref import FileRef
# from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
# from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
# from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType

path = 'C://Users//nhagl//Desktop//work//demo//ComplianceTool//adobe_api//pdfs'
dir_list = os.listdir(path)

for pdf_name in dir_list:
    # zip_file = "./ExtractTextFrom" +pdf_name.replace(".pdf","") + ".zip"
    # input_pdf = "./pdfs/" + pdf_name

    # if not os.path.isfile(zip_file):
    #     #Initial setup, create credentials instance.
    #     credentials = Credentials.service_account_credentials_builder()\
    #         .from_file("./pdfservices-api-credentials.json") \
    #         .build()

    #     #Create an ExecutionContext using credentials and create a new operation instance.
    #     execution_context = ExecutionContext.create(credentials)

    #     extract_pdf_operation = ExtractPDFOperation.create_new()

    #     #Set operation input from a source file.
    #     source = FileRef.create_from_local_file(input_pdf)
    #     extract_pdf_operation.set_input(source)

    #     #Build ExtractPDF options and set them into the operation
    #     extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
    #         .with_element_to_extract(ExtractElementType.TEXT) \
    #         .build()
    #     extract_pdf_operation.set_options(extract_pdf_options)

    #     #Execute the operation.
    #     result: FileRef = extract_pdf_operation.execute(execution_context)

    #     #Save the result to the specified location.
    #     result.save_as(zip_file)

    # archive = zipfile.ZipFile(zip_file, 'r')
    # jsonentry = archive.open('structuredData.json')
    # jsondata = jsonentry.read()
    # data = json.loads(jsondata)

    # header_list = []
    # for element in data["elements"]:
    #     if("TOC" in element["Path"]) or ("H" in element["Path"]):
    #         try:
    #             header_list.append(element["Text"])
    #         except:
    #             continue
    
    # pattern = re.compile(r'\s+')
    # reduced_string = ""
    # reduced_list = []
    # for header in header_list:
    #     reduced_string += re.sub(pattern, '', header)
    #     reduced_list.append(re.sub(pattern, '', header))

    doc = fitz.open("./pdfs/" + pdf_name) # open a supported document
    index = 0
    whole_document = []
    while index < doc.page_count:
        page = doc[index] # load the required page (0-based index)
        whole_document.append(page.get_text("dict", sort=True))
        index += 1
        # text = page.get_text("dict", sort=True) # extract plain text
        # # print(text) # process or print it:
        # print(json.dumps(text, indent=4)
    final_list = []
    for page in whole_document:
        page_width = page['width']
        page_height = page['height']
        for i in page['blocks']:
            try: 
                for line in i['lines']:

                    text_line = ""
                    span_list = []
                    font_size = 0
                    font_family = ""
                    font_weight = 0
                    page_number = whole_document.index(page) + 1
                    x_origin = line['bbox'][0]
                    x_end = line['bbox'][2]
                    y_origin = line['bbox'][1]
                    y_end = line['bbox'][3]
                    # header = 0
                    for span in line['spans']:
                        try:
                            text_line += span['text']
                            span_list.append(span['text'])
                        except:
                            continue
                        try:
                            if span["size"] > font_size:
                                font_size = span["size"]
                        except:
                            continue
                        try:
                            if "Bold" in span["font"]:
                                font_weight = 1
                                font_family = span["font"]
                            elif "Underline" in span["font"]:
                                font_weight = 1
                                font_family = span["font"]
                            elif "Ital" in span["font"]:
                                font_weight = 1
                                font_family = span["font"]
                            elif ("Bold" not in font_family) and ("Underline" not in font_family) and ("Italic" not in font_family):
                                font_family = span["font"]
                        except:
                            continue
                    # pattern = re.compile(r'\s+')
                    # if re.sub(pattern, '', text_line) in reduced_list:
                    #     header = 1
                    # else:
                    #     for item in span_list:
                    #         if re.sub(pattern, '', item) in reduced_list:
                    #             header = 1
                    if text_line.isspace() or text_line == "":
                        continue
                    else:
                        final_list.append({
                            'Text Line' : text_line.encode('utf8'),
                            'Font Size' : font_size,
                            'Font Family' : font_family,
                            'Font Weight': font_weight,
                            'Page Number': page_number,
                            'X Origin': x_origin,
                            'X End': x_end,
                            'Y Origin': y_origin,
                            'Y End': y_end,
                            'Page Width': page_width,
                            'Page Height': page_height,
                            'Document': pdf_name
                        })
            except:
                continue

    keys = final_list[0].keys()

    with open('test.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        # dict_writer.writeheader()
        dict_writer.writerows(final_list)

In [2]:
# df = pd.DataFrame()
# df = df.append({
#     'Text Line' : 'Stuff', #complete text line
#     'Font Size' : 30, #max occurence of font size in line
#     'Font Family' : 50, #max occurence of font family in line
#     'Font Weight': 1, #max occurence of font weight in line
#     'Page Number': 1, #index + 1 of line
#     'X Origin': 1, #['bbox'][0] for line
#     'X End': 2, #['bbox'][2] for line
#     'Y Origin': 1, #['bbox'][1] for line
#     'Y End': 1, #['bbox'][3] for line
#     'Page Width': 20,
#     'Page Height': 25
#     },ignore_index = True)